In [1]:
!pip install torch transformers

import torch
import torch.nn as nn # Pytorch Neural Network
import pandas as pd # 데이터 분석하기
from torch.utils.data import Dataset, DataLoader # 데이터 가져오고 처리하기
from torch.nn.utils.rnn import pad_sequence # 시퀀스 패딩 사용 이유 ? 데이터 관련하여 크기가 일관 X 라서 처리 따로 하기
from transformers import DistilBertTokenizer, DistilBertModel # DistilBert 모델 활용

In [2]:
# 훈련 데이터 가져오기

train_data_folder = '/kaggle/input/nlp-getting-started/train.csv'
train_data = pd.read_csv(train_data_folder)

# Distil Bert의 토크나이저 및 모델 가져오기

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

# 텍스트 데이터를 Distil Bert의 입력 형식으로 전환

def preprocess_text(text):
    inputs = tokenizer(text, truncation=True, padding=True, max_length=128)
    return inputs

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [3]:
# 사용자 정의 데이터셋

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data['text'].iloc[idx]
        label = self.data['target'].iloc[idx]
        inputs = self.tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        return {'input_ids': inputs['input_ids'].squeeze(),
                'attention_mask': inputs['attention_mask'].squeeze(),
                'label': torch.tensor(label, dtype=torch.float32)}

# 데이터셋과 데이터 로더 생성하기

train_dataset = CustomDataset(train_data, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [4]:
# 모델 구성하기

class DistilBERTClassifier(nn.Module):
    def __init__(self, pretrained_model_name='distilbert-base-uncased', hidden_size=768, num_classes=1, dropout_prob=0.1):
        super(DistilBERTClassifier, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained(pretrained_model_name)
        self.fc1 = nn.Linear(hidden_size, 256)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(256, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        pooled_output = torch.mean(last_hidden_state, dim=1)
        x = self.fc1(pooled_output)
        x = self.dropout(x)
        x = self.fc2(x)
        logits = self.sigmoid(x)
        return logits

In [5]:
# 모델, 손실 함수, 옵티마이저 생성하기

model = DistilBERTClassifier()
criterion = nn.BCEWithLogitsLoss()  # BCELoss와 Sigmoid를 결합한 함수 사용 완료
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# GPU로 이동하기

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

DistilBERTClassifier(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin

In [6]:
# 입력 데이터 크기 조정(DataLoader의 미니 배치 크기 문제로 인해 조정)

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = torch.tensor([item['label'] for item in batch], dtype=torch.float32)  # Label을 tensor로 변환

    # Dynamic Padding을 사용하여 각 샘플을 동적으로 패딩
    padded_input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    padded_attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)

    return {'input_ids': padded_input_ids, 'attention_mask': padded_attention_mask, 'label': labels}

# DataLoader에서 collate_fn 사용

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, collate_fn=collate_fn)

In [7]:
# 훈련 하기

epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {average_loss:.4f}')

Epoch 1/5, Loss: 0.6907
Epoch 2/5, Loss: 0.6164
Epoch 3/5, Loss: 0.6018
Epoch 4/5, Loss: 0.5962
Epoch 5/5, Loss: 0.5919


In [8]:
# 테스트 데이터 불러오기

test_data_folder = '/kaggle/input/nlp-getting-started/test.csv'
test_data = pd.read_csv(test_data_folder)

# 테스트 입력 데이터 크기 조정(DataLoader의 미니 배치 크기 문제로 인해 조정)

def collate_fn_test(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]

    # Dynamic Padding을 사용하여 각 샘플을 동적으로 패딩
    padded_input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    padded_attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)

    collated_batch = {'input_ids': padded_input_ids, 'attention_mask': padded_attention_mask}

    return collated_batch

In [9]:
# 테스트 데이터와 토크나이저를 사용해서 테스트 입력 형식에 맞게 제공하기

class TestDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data['text'].iloc[idx]
        inputs = self.tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
        return {'input_ids': inputs['input_ids'].squeeze(),
                'attention_mask': inputs['attention_mask'].squeeze()}

# 테스트할 때도 사용자 정의 해주기

test_dataset = TestDataset(test_data, tokenizer)

# 테스트 데이터 - DataLoader 불러오기

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn_test)

In [10]:
# 모델을 평가 모드로 설정하기

model.eval()

# 테스트 세트에서 예측을 수행하고 결과를 저장하기

predictions = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask)
        logits = outputs.squeeze().cpu().numpy()
        predictions.extend(logits)

test_ids = test_data['id'].tolist()

submission_df = pd.DataFrame({'id': test_ids, 'target': predictions})

# DataFrame을 CSV 파일로 저장하기
submission_df.to_csv('submission.csv', header=False)